## Using the titanic data, in your classification-exercises repository, create a notebook, decision_tree.ipynb where you will do the following:

In [60]:
import pydataset
from env import get_db_url

import pandas as pd
import numpy as np

from prepare import prep_titanic
from prepare import titanic_split

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

#import graphviz
#from graphviz import Graph

In [2]:
df = prep_titanic()
df['baseline_prediction'] = 0
df.head()

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embarked,alone,sex_male,embarked_Q,embarked_S,baseline_prediction
0,0,0,3,male,1,0,7.2500,S,0,1,0,1,0
1,1,1,1,female,1,0,71.2833,C,0,0,0,0,0
2,2,1,3,female,0,0,7.9250,S,1,0,0,1,0
3,3,1,1,female,1,0,53.1000,S,0,0,0,1,0
4,4,0,3,male,0,0,8.0500,S,1,1,0,1,0


In [3]:
df, train, validate, test = titanic_split(df)

<hr style="border: 2px solid #000000;">

In [4]:
# inspect size
train.shape, validate.shape, test.shape

((534, 13), (178, 13), (179, 13))

In [5]:
# inspect
train.info(), validate.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 455 to 496
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   passenger_id         534 non-null    int64  
 1   survived             534 non-null    int64  
 2   pclass               534 non-null    int64  
 3   sex                  534 non-null    object 
 4   sibsp                534 non-null    int64  
 5   parch                534 non-null    int64  
 6   fare                 534 non-null    float64
 7   embarked             534 non-null    object 
 8   alone                534 non-null    int64  
 9   sex_male             534 non-null    uint8  
 10  embarked_Q           534 non-null    uint8  
 11  embarked_S           534 non-null    uint8  
 12  baseline_prediction  534 non-null    int64  
dtypes: float64(1), int64(7), object(2), uint8(3)
memory usage: 47.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 176 to 

(None, None, None)

### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [6]:
df.head()

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embarked,alone,sex_male,embarked_Q,embarked_S,baseline_prediction
0,0,0,3,male,1,0,7.2500,S,0,1,0,1,0
1,1,1,1,female,1,0,71.2833,C,0,0,0,0,0
2,2,1,3,female,0,0,7.9250,S,1,0,0,1,0
3,3,1,1,female,1,0,53.1000,S,0,0,0,1,0
4,4,0,3,male,0,0,8.0500,S,1,1,0,1,0


### Chekcing to the count for survived passengers. 
* The value 0 means False and 1 means True.

In [7]:
df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [8]:
df.head(1)

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embarked,alone,sex_male,embarked_Q,embarked_S,baseline_prediction
0,0,0,3,male,1,0,7.25,S,0,1,0,1,0


## The baseline accuracy:

In [9]:
# checking the baseline accuracy
baseline_accuracy = (df.survived == df.baseline_prediction).mean()
baseline_accuracy

0.6161616161616161

<hr style="border: 2px solid #000000;">

In [10]:
df.columns

Index(['passenger_id', 'survived', 'pclass', 'sex', 'sibsp', 'parch', 'fare',
       'embarked', 'alone', 'sex_male', 'embarked_Q', 'embarked_S',
       'baseline_prediction'],
      dtype='object')

In [11]:
'pclass', 'sex', 'sibsp', 'parch', 'fare', 'alone', 'sex_male', 

('pclass', 'sex', 'sibsp', 'parch', 'fare', 'alone', 'sex_male')

In [12]:
df.dtypes

passenger_id             int64
survived                 int64
pclass                   int64
sex                     object
sibsp                    int64
parch                    int64
fare                   float64
embarked                object
alone                    int64
sex_male                 uint8
embarked_Q               uint8
embarked_S               uint8
baseline_prediction      int64
dtype: object

In [13]:
df['fare'] = df['fare'].astype('float64')

<hr style="border: 2px solid #000000;">

In [14]:
# create X & y version of train/validate/test
# where X contains the features we want to use and y is a series with just the target variable

X_train = train.drop(columns=['passenger_id', 'survived', 'sex', 'embarked', 'embarked_Q', 
        'embarked_S', 'baseline_prediction'])
y_train = train.survived

In [15]:
X_validate = validate.drop(columns=['passenger_id', 'survived', 'sex', 'embarked', 'embarked_Q', 
        'embarked_S', 'baseline_prediction'])
y_validate = validate.survived

In [16]:
X_test = test.drop(columns=['passenger_id', 'survived', 'sex', 'embarked', 'embarked_Q', 
        'embarked_S', 'baseline_prediction'])
y_test = test.survived

<hr style="border: 2px solid #000000;">

In [17]:
#Compare df columns

print(f"""train:
{train.columns.to_list()} 
_____________________________________________
X_train:
{X_train.columns.to_list()}
_____________________________________________
X_validate:
{X_validate.columns.to_list()}
""")

train:
['passenger_id', 'survived', 'pclass', 'sex', 'sibsp', 'parch', 'fare', 'embarked', 'alone', 'sex_male', 'embarked_Q', 'embarked_S', 'baseline_prediction'] 
_____________________________________________
X_train:
['pclass', 'sibsp', 'parch', 'fare', 'alone', 'sex_male']
_____________________________________________
X_validate:
['pclass', 'sibsp', 'parch', 'fare', 'alone', 'sex_male']



### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [18]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [19]:
clf = DecisionTreeClassifier(max_depth=3, random_state=123)

In [20]:
# model.fit(X, y)

clf = clf.fit(X_train, y_train)

In [34]:
# make prediction on train obeservations

y_pred = clf.predict(X_train)
y_pred[0:5]

array([0, 1, 0, 0, 0])

In [35]:
#estimates probabilty

y_pred_proba = clf.predict_proba(X_train)
y_pred_proba[0:5]

array([[0.905     , 0.095     ],
       [0.01785714, 0.98214286],
       [0.69105691, 0.30894309],
       [0.69105691, 0.30894309],
       [0.905     , 0.095     ]])

In [36]:
#compute the accuracy

print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.82


### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [52]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [39]:
# confusion matrix

confusion_matrix(y_train, y_pred)

array([[295,  34],
       [ 63, 142]])

In [40]:
#target value count

y_train.value_counts()

0    329
1    205
Name: survived, dtype: int64

In [55]:
# 0=Did Not Survive 
# 1=Survived



pd.DataFrame(confusion_matrix(y_train, y_pred), index=['Ac_False', 'Ac_True'], columns=['P_False', 'P_True'])

,P_False,P_True
Ac_False,295,34
Ac_True,63,142


In [46]:
y_train.value_counts()

0    329
1    205
Name: survived, dtype: int64

In [53]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86       329
           1       0.81      0.69      0.75       205

    accuracy                           0.82       534
   macro avg       0.82      0.79      0.80       534
weighted avg       0.82      0.82      0.82       534



### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

### 5. Run through steps 2-4 using a different max_depth value.

### 6. Which model performs better on your in-sample data?

### 7. Which model performs best on your out-of-sample data, the validate set?